# Search Property Listings
1. Model Setting
2. Create API filter from user's query
3. Get Property Listings from Zillow API
4. Generate advice with the property listings

## 1. Model Setting - Gemini pro 1.0

In [4]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core

In [5]:
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

2024-05-07 22:05:56.764255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 22:05:57.936525: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-07 22:05:57.936651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [7]:
vertexai.init(project="adsp-capstone-property-pilot", location="us-central1")

In [70]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

### Test

In [71]:
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello there, how can I assist you today? 😊


## 2. Create API filter from user's query

In [10]:
def generate_prompt_apifilter(instruction, user_query):
    return instruction.replace("{USER_QUERY}", user_query)

In [14]:
instruction = """
### Instructions ###
I want you to act as a principal software engineer. You'll be given a task to determine the parameters of an API call based on the "User's query" below, 
please modify the conditions and create the appropriate API filter. Provide your answer only for the part within ‘{}’. 
Do not include any other explanations in your response. Read the below instructions.

— Filter Options (Optional, so not required if not specified in the User's query) —

1. location
Format: STRING(lowercase)
Example: santa monica, ca

*If not specifically mentioned, use 'chicago, il'.
*Do not alter the format from the above example. 
*If you want to search for a specific neighborhood like Chicago's West Loop, please specify it in the Filter option '16. keywords', not here.
Incorrect example: "location": "west loop, chicago, il"
Correct example: "location": "chicago, il", "keywords": "west loop"


2. status_type
For purchase or for rent
Format: STRING
Example:
ForSale
ForRent

3. home_type
Property type comma-separated or empty for all types
Format: STRING
Example(ForRent):
Townhomes
Houses
Apartments_Condos_Co-ops

Example(ForSale):
Multi-family
Apartments
Houses
Manufactured
Condos
LotsLand
Townhomes

*If it cannot be determined from the User's query, specification is not necessary.

4. minPrice
If status_type = ForSale you can filter by min price.
Format: NUMBER
Example: 200000

5. maxPrice
If status_type = ForSale you can filter by max price.
Format: NUMBER
Example: 900000

6. rentMinPrice
If status_type = ForRent you can filter by min rent price.
Format: NUMBER
Example: 2000

7. rentMaxPrice
If status_type = ForRent you can filter by max rent price.
Format: NUMBER
Example: 3000

8. bathsMin
Bathrooms min count
Format: NUMBER
Example: 2
* If a specific number of baths is specified, rather than a range, please enter the same number for both bathsMin and bathsMax.

9. bathsMax
Bathrooms max count
Format: NUMBER
Example: 3

10. bedsMin
Bedrooms min count
Format: NUMBER
Example: 1
* If a specific number of bedrooms is specified, rather than a range, please enter the same number for both bedsMin and bedsMax.

11. bedsMax
Bedrooms max count
Format: NUMBER
Example: 3

12. sqftMin
Square Feet min value
Format: NUMBER
Example: 600

13. sqftMax
Square Feet max value
Format: NUMBER
Example: 1500

14. buildYearMin
Year Built min value
Format: NUMBER
Example: 1980

15. buildYearMax
Year Built max value
Format: NUMBER
Example: 2023

16. keywords
Filter with keywords.
Format: STRING
Example1: ‘gym’
Example2: ‘West Loop’
Example3: ‘pet, pool, South Loop‘
*Not necessary if already specified in another filter or if there is no clear specification in user’s query. Specify conditions that cannot be specified in other filters by using keywords.

17. sort
Sorting criteria for the results
Always specify "Newest"


Now, use the following query to determine the appropriate parameters.

### Example query ###
I am looking for a one-bedroom.

### Example Output ###
{
"location":"chicago, il",
"bedsMin":"1",
"bedsMax":"1",
"sort":"Newest"
}

### Example query ###
I am looking for a rental in South Loop, Chicago, with at least one bedroom and a rent of $3000 or less per month.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"rentMaxPrice": 3000,
"bedsMin": 1,
"keywords": "South Loop",
"sort":"Newest"
}

### Example query ###
I am looking for a rental in West Loop, Chicago, with one bedroom and a rent between $2000 and $2500.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"rentMinPrice": 2500,
"rentMaxPrice": 3000,
"bedsMin": 1,
"bedsMax": 1,
"keywords": "West Loop",
"sort":"Newest"
}

### Example query ###
I want to buy a house in River North with at least two bedrooms.

### Example Output ###
{
"location":"chicago, il",
"status_type": "ForSale",
"keywords":"River North",
"bedsMin":"2",
"sort":"Newest"
}

### Example query ###
I am looking for a rental around Wicker Park in Chicago with a gym and a pool. It should be built within the last 10 years.

### Example Output ###
{
"location": "chicago, il",
"home_type": "Apartments_Condos_Co-ops",
"status_type": "ForRent",
"buildYearMin": 2014,
"keywords": "Wicker Park, gym, pool",
"sort":"Newest"
}

### Example query ###
I am looking for a rental with a balcony, over 700 sqft in size, and less than 5 years old, located near downtown.

### Example Output ###
{
"location": "chicago, il",
"status_type": "ForRent",
"sqftMin":"700",
"buildYearMin": 2019,
"keywords": "downtown, balcony",
"sort":"Newest"
}


### User’s query ###
{USER_QUERY}

### Output ###
"""

In [15]:
user_query = "I am looking for a two-bedroom."
print(generate_prompt_apifilter(instruction, user_query))


### Instructions ###
I want you to act as a principal software engineer. You'll be given a task to determine the parameters of an API call based on the "User's query" below, 
please modify the conditions and create the appropriate API filter. Provide your answer only for the part within ‘{}’. 
Do not include any other explanations in your response. Read the below instructions.

— Filter Options (Optional, so not required if not specified in the User's query) —

1. location
Format: STRING(lowercase)
Example: santa monica, ca

*If not specifically mentioned, use 'chicago, il'.
*Do not alter the format from the above example. 
*If you want to search for a specific neighborhood like Chicago's West Loop, please specify it in the Filter option '16. keywords', not here.
Incorrect example: "location": "west loop, chicago, il"
Correct example: "location": "chicago, il", "keywords": "west loop"


2. status_type
For purchase or for rent
Format: STRING
Example:
ForSale
ForRent

3. home_type
Prope

### Test

In [16]:
user_query = "I am looking for a two-bedroom."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"bedsMin": "2",
"bedsMax": "2"
}


In [17]:
user_query = "I am looking for a rental in Fulton Market, Chicago, with at least one bedroom and a rent of $3500 or less per month."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
    "location":"chicago, il",
    "status_Type":"ForRent",
    "rentMaxPrice": 3500,
    "bedsMin": 1,
    "keywords":"Fulton Market",
    "sort": "Newest"
}


In [18]:
user_query = "I want to buy a house in Chinatown, Chicago with a minimum of 2 bathrooms and 2 bedrooms."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"status_type": "ForSale",
"keywords":"Chinatown",
"bedsMin":"2",
"bathsMin":"2",
"sort":"Newest"
}


In [19]:
user_query = "I am looking for a rental around Hyde Park with a gym and a pool. It should be built within the last 15 years."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"home_type": "Apartments_Condos_Co-ops",
"status_type": "ForRent",
"buildYearMin": 2009,
"keywords":"Hyde Park, gym, pool",
"sort":"Newest"
}


In [20]:
user_query = "I am looking for a rental in the Old Town area of Chicago with a balcony and a pool."
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"status_type":"ForRent",
"keywords":"Old Town, balcony, pool",
"sort":"Newest"
}


## 3. Get Property Listings from Zillow API

### Example - Extended Search endpoint

In [76]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": "RAPID_API_KEY",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

In [22]:
querystring = {"location":"chicago, il", "status_type":"ForRent", "bathsMin":"1","bedsMin":"3", "keywords":"west", "sort":"Newest"}
response = requests.get(url, headers = headers, params = querystring)

In [23]:
response.json()

{'props': [{'dateSold': None,
   'propertyType': 'APARTMENT',
   'lotAreaValue': None,
   'address': '2253 W Iowa St #3F, Chicago, IL 60622',
   'variableData': None,
   'unit': '# 3F',
   'zestimate': None,
   'imgSrc': 'https://photos.zillowstatic.com/fp/24cfac4e50b54ac036925874a4d4b395-p_e.jpg',
   'price': 2695,
   'detailUrl': '/homedetails/2253-W-Iowa-St-3F-Chicago-IL-60622/2069892388_zpid/',
   'bedrooms': 3,
   'contingentListingType': None,
   'longitude': -87.68405,
   'latitude': 41.897385,
   'listingStatus': 'FOR_RENT',
   'zpid': '2069892388',
   'listingSubType': {},
   'rentZestimate': 2625,
   'daysOnZillow': 0,
   'bathrooms': 1,
   'livingArea': 1200,
   'country': 'USA',
   'currency': 'USD',
   'lotAreaUnit': None,
   'hasImage': True},
  {'units': [{'roomForRent': False, 'beds': '3', 'price': '$7,113+'}],
   'listingStatus': 'FOR_RENT',
   'zpid': '41.86681--87.62358',
   'longitude': -87.62358,
   'buildingName': 'NEMA Chicago',
   'address': '1210 S Indiana Ave,

In [24]:
def extract_properties(response, n, fields):
    """
    Extracts and returns the top 'n' properties from a given API response
    based on the specified fields, appending the base URL to 'detailUrl'.
    
    Parameters:
    - response: The response object from an API call.
    - n: The number of top properties to extract.
    - fields: A list of strings representing the fields to extract from each property.
    
    Returns:
    - A list of dictionaries, each representing a property with only the specified fields.
    """
    base_url = "https://www.zillow.com"
    try:
        # Convert the response to JSON format if it's not already a dictionary
        if not isinstance(response, dict):
            response = response.json()

        # Extract the top 'n' properties based on the specified fields
        top_properties = []
        for prop in response['props'][:n]:
            extracted_prop = {field: (base_url + prop['detailUrl'] if field == 'detailUrl' else prop.get(field, None)) for field in fields}
            top_properties.append(extracted_prop)

        return top_properties

    except Exception as e:
        print("An error occurred:", e)
        return []  # Return an empty list in case of an error


In [25]:
# Define fields to extract
fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]

In [26]:
# Get top 3 properties with selected fields
top_properties = extract_properties(response, 3, fields)
top_properties

[{'propertyType': 'APARTMENT',
  'address': '2253 W Iowa St #3F, Chicago, IL 60622',
  'price': 2695,
  'bedrooms': 3,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/2253-W-Iowa-St-3F-Chicago-IL-60622/2069892388_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/24cfac4e50b54ac036925874a4d4b395-p_e.jpg',
  'longitude': -87.68405,
  'latitude': 41.897385},
 {'propertyType': None,
  'address': '1210 S Indiana Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/nema-chicago/9RwqTm/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/35f6b1437825b0cf43201316158e8000-p_e.jpg',
  'longitude': -87.62358,
  'latitude': 41.86681},
 {'propertyType': 'APARTMENT',
  'address': '2065 N. Hoyne Ave., 2065 N Hoyne Ave #3R, Chicago, IL 60647',
  'price': 2295,
  'bedrooms': 3,
  'bathrooms': 2,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/2065-n.-hoyne-ave./5XgKsR/',
  'imgSrc'

### Mapping the Properties

In [27]:
#pip install folium

In [28]:
import folium

def create_property_map(properties):
    # Attribution for custom tileset
    attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    
    # Initialize the map centered around Chicago
    property_map = folium.Map(location=[41.881832, -87.623177], zoom_start=11, tiles='OpenStreetMap_Mapnik', attr=attr)

    # Loop through each property in the list
    for prop in properties:
        # Extract latitude and longitude
        latitude = prop['latitude']
        longitude = prop['longitude']
        detail_url = prop['detailUrl']
        address = prop['address']

        # Add a marker to the map
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(f'<a href="{detail_url}" target="_blank">{address}</a>', max_width=250),
            tooltip="Click for Zillow Listing",
            icon=folium.Icon(icon = 'home', color="blue")
        ).add_to(property_map)

    # Return the map
    return property_map

In [29]:
# Create the map
property_map = create_property_map(top_properties)

display(property_map)

### Retrieving Restaurant Info Using Yelp API

In [75]:
yelp_api_key = 'YELP_API_KEY'

### Extract restaurant category if there is one specified in the user query

In [31]:
def generate_prompt_rest_category(user_query):
    instruction = f"""
    ### Instructions ###
    Identify the category of restaurant the user is interested in based on their query. 
    If the query doesn't specify a type, return "restaurants". 
    Output should be a lowercase string. 
    If there are multiple categories, separate them with a comma with NO WHITE SPACES OR NEW LINE CHARACTERS.

    ### User’s query ###
    {user_query}

    ### Output ###
    """
    return instruction

### Test restaurant category function

In [32]:
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month. What are the best bars nearby?"

# Example call to a language model (you need to implement get_chat_response or use an existing function that sends this prompt to a language model)
prompt = generate_prompt_rest_category(user_query)
categories = get_chat_response(chat, prompt)  # Ensure this function captures only the output after "### Output ###"

print(categories)

bars


In [33]:
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month. What are the best french restaurants and coffee shops nearby?"

# Example call to a language model (you need to implement get_chat_response or use an existing function that sends this prompt to a language model)
prompt = generate_prompt_rest_category(user_query)
categories = get_chat_response(chat, prompt)  # Ensure this function captures only the output after "### Output ###"

print(categories)

french,coffee


In [34]:
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month."

# Example call to a language model (you need to implement get_chat_response or use an existing function that sends this prompt to a language model)
prompt = generate_prompt_rest_category(user_query)
categories = get_chat_response(chat, prompt)  # Ensure this function captures only the output after "### Output ###"

print(categories)

restaurants


### Call Yelp API Functions

In [35]:
def get_top_rated_businesses(api_key, location, category, top_n=2):
    """Fetches top-rated businesses near a given location from Yelp."""
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    params = {
        'location': location,
        'term': 'restaurants',
        'radius': 2000,
        'categories': category,
        'sort_by': 'rating',
        'limit': top_n
    }
    print(params)
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [36]:
def fetch_top_businesses_near_properties(api_key, properties, user_query, chat):
    """
    Fetches top-rated businesses of a specified category near each property in a given list,
    using a category inferred from a user query processed by Gemini LLM.
    """
    prompt = generate_prompt_rest_category(user_query)
    category = get_chat_response(chat, prompt)  # Get the category from the LLM
    category = category.replace('"', '')  # Strip any quotation marks from the category
    if not category or category.lower() == "restaurants":
        category = "restaurants"  # Fallback to a default category if none specified
    print(f"Category determined: {category}")  # Log the category for debugging

    top_restaurants_yelp = []
    for property in properties:
        location = f"{property['latitude']}, {property['longitude']}"
        top_businesses = get_top_rated_businesses(api_key, location, category)
        top_restaurants_yelp.append({
            'address': property['address'],
            'top_businesses': top_businesses
        })
    
    return top_restaurants_yelp

### Test Category Searching on Yelp

In [37]:
# Call the function
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month. What are the best Mexican restaurants nearby?"

yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, chat)

for result in yelp_results:
    print(result)

Category determined: mexican
{'location': '41.897385, -87.68405', 'term': 'restaurants', 'radius': 2000, 'categories': 'mexican', 'sort_by': 'rating', 'limit': 2}
{'location': '41.86681, -87.62358', 'term': 'restaurants', 'radius': 2000, 'categories': 'mexican', 'sort_by': 'rating', 'limit': 2}
{'location': '41.91961, -87.67988', 'term': 'restaurants', 'radius': 2000, 'categories': 'mexican', 'sort_by': 'rating', 'limit': 2}
{'address': '2253 W Iowa St #3F, Chicago, IL 60622', 'top_businesses': {'businesses': [{'id': 'HLUuqIzYaituoifAZTsiqw', 'alias': 'los-3-compadres-de-guerrero-chicago', 'name': 'Los 3 Compadres De Guerrero', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/MF9NGkyv6wdn2YnKHhTc6A/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/los-3-compadres-de-guerrero-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q', 'review_count': 6, 'categories': [{'alias': 'tacos', 'ti

In [38]:
# Call the function
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month. What are the best bars nearby?"

yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, chat)

for result in yelp_results:
    print(result)

Category determined: bars
{'location': '41.897385, -87.68405', 'term': 'restaurants', 'radius': 2000, 'categories': 'bars', 'sort_by': 'rating', 'limit': 2}
{'location': '41.86681, -87.62358', 'term': 'restaurants', 'radius': 2000, 'categories': 'bars', 'sort_by': 'rating', 'limit': 2}
{'location': '41.91961, -87.67988', 'term': 'restaurants', 'radius': 2000, 'categories': 'bars', 'sort_by': 'rating', 'limit': 2}
{'address': '2253 W Iowa St #3F, Chicago, IL 60622', 'top_businesses': {'businesses': [{'id': '0ucCbCekcn5XdzvRXnHl2w', 'alias': 'desert-hawk-chicago', 'name': 'Desert Hawk', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/Wvt0tJNb-HbxRnBQ7iUUPQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/desert-hawk-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q', 'review_count': 23, 'categories': [{'alias': 'tradamerican', 'title': 'American'}, {'alias': 'bars', 'title': 'Bars'

In [39]:
# Call the function
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month. What are the best french restaurants and coffee shops nearby?"

yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, chat)

for result in yelp_results:
    print(result)

Category determined: french,coffee
{'location': '41.897385, -87.68405', 'term': 'restaurants', 'radius': 2000, 'categories': 'french,coffee', 'sort_by': 'rating', 'limit': 2}
{'location': '41.86681, -87.62358', 'term': 'restaurants', 'radius': 2000, 'categories': 'french,coffee', 'sort_by': 'rating', 'limit': 2}
{'location': '41.91961, -87.67988', 'term': 'restaurants', 'radius': 2000, 'categories': 'french,coffee', 'sort_by': 'rating', 'limit': 2}
{'address': '2253 W Iowa St #3F, Chicago, IL 60622', 'top_businesses': {'businesses': [{'id': 'gxUFFsSLa-IyouuRD2Ma6w', 'alias': 'spinning-j-bakery-and-soda-fountain-chicago-4', 'name': 'Spinning J Bakery and Soda Fountain', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/CVtjtq4_rYRd7J-qNk9R-A/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/spinning-j-bakery-and-soda-fountain-chicago-4?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q', 'rev

In [40]:
user_query = "I am looking for an apartment in the Hyde Park neighborhood of Chicago. I want two bedrooms and at least one bathroom. Rent should be less than $1500 per month."

yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, chat)

for result in yelp_results:
    print(result)

Category determined: restaurants
{'location': '41.897385, -87.68405', 'term': 'restaurants', 'radius': 2000, 'categories': 'restaurants', 'sort_by': 'rating', 'limit': 2}
{'location': '41.86681, -87.62358', 'term': 'restaurants', 'radius': 2000, 'categories': 'restaurants', 'sort_by': 'rating', 'limit': 2}
{'location': '41.91961, -87.67988', 'term': 'restaurants', 'radius': 2000, 'categories': 'restaurants', 'sort_by': 'rating', 'limit': 2}
{'address': '2253 W Iowa St #3F, Chicago, IL 60622', 'top_businesses': {'businesses': [{'id': 'hva3YCKzqhtkKerQWsXo7Q', 'alias': 'janellie-s-kitchen-chicago-2', 'name': 'Janellie’s Kitchen', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/gA6saQlTNJu03XZuXwafdg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/janellie-s-kitchen-chicago-2?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q', 'review_count': 12, 'categories': [{'alias': 'puertorican', 'ti

### Extract Restaurant Info

In [41]:
def extract_business_info(addresses, n, fields):
    """
    Extracts and returns information from a given list of addresses, each containing businesses,
    based on the specified fields and the top 'n' businesses per address.
    
    Parameters:
    - addresses: A list of dictionaries, each representing an address with nested businesses.
    - n: The number of top businesses to extract from each address.
    - fields: A list of strings representing the fields to extract from each business.
    
    Returns:
    - A list of dictionaries, each representing a business with only the specified fields.
    """
    extracted_businesses = []
    try:
        # Loop through each address entry
        for address_entry in addresses:
            # Extract top 'n' businesses
            for business in address_entry['top_businesses']['businesses'][:n]:
                # Extract specified fields, handling nested structures like 'categories' and 'location'
                extracted_business = {
                    'business_name': business.get('name'),
                    'categories_titles': [category['title'] for category in business.get('categories', [])],
                    'rating': business.get('rating'),
                    'latitude': business.get('coordinates', {}).get('latitude'),
                    'longitude': business.get('coordinates', {}).get('longitude'),
                    'display_address': " ".join(business.get('location', {}).get('display_address', [])),
                    'url': business.get('url')
                }
                # Only append fields that were specifically requested
                filtered_business = {field: extracted_business[field] for field in fields if field in extracted_business}
                extracted_businesses.append(filtered_business)

        return extracted_businesses

    except Exception as e:
        print("An error occurred:", e)
        return []  # Return an empty list in case of an error

In [42]:
# Example usage

fields_to_extract = ['business_name', 'categories_titles', 'rating', 'latitude', 'longitude', 'display_address', 'url']
top_restaurants = extract_business_info(yelp_results, 2, fields_to_extract)
for info in top_restaurants:
    print(info)

{'business_name': 'Janellie’s Kitchen', 'categories_titles': ['Puerto Rican'], 'rating': 5.0, 'latitude': 41.90281613454157, 'longitude': -87.69265859457461, 'display_address': '2617 W Division St Chicago, IL 60622', 'url': 'https://www.yelp.com/biz/janellie-s-kitchen-chicago-2?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q'}
{'business_name': 'Healthy Substance Cantina', 'categories_titles': ['Vegan'], 'rating': 5.0, 'latitude': 41.908631, 'longitude': -87.675257, 'display_address': '1480 N Milwaukee Ave Chicago, IL 60622', 'url': 'https://www.yelp.com/biz/healthy-substance-cantina-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q'}
{'business_name': 'La Bodega', 'categories_titles': ['Caterers', 'Food Delivery Services', 'Tapas/Small Plates'], 'rating': 5.0, 'latitude': 41.86857605, 'longitude': -87.62597656, 'd

### Combine Restaurant and Property Info

In [43]:
def merge_property_and_restaurant_info(properties, restaurants):
    """
    Merges each property with its corresponding top-rated restaurants.

    Parameters:
    - properties: A list of dictionaries, each representing a property.
    - restaurants: A list of dictionaries, each representing top-rated restaurants near the properties.

    Returns:
    - A list of dictionaries, each representing a property merged with restaurant information.
    """
    # Ensure there are two restaurants for each property, assuming a structure of 1:2 mapping
    if len(restaurants) != 2 * len(properties):
        raise ValueError("Each property must correspond to exactly two restaurants.")

    merged_data = []
    for i, property in enumerate(properties):
        # Merge the property dict with its corresponding restaurants' info
        restaurant_info = {
            "restaurant_1": restaurants[2*i],
            "restaurant_2": restaurants[2*i + 1]
        }
        # Merge two dictionaries and add to the list
        merged_property = {**property, **restaurant_info}
        merged_data.append(merged_property)

    return merged_data

In [44]:
top_properties = merge_property_and_restaurant_info(top_properties, top_restaurants)
top_properties

[{'propertyType': 'APARTMENT',
  'address': '2253 W Iowa St #3F, Chicago, IL 60622',
  'price': 2695,
  'bedrooms': 3,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/2253-W-Iowa-St-3F-Chicago-IL-60622/2069892388_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/24cfac4e50b54ac036925874a4d4b395-p_e.jpg',
  'longitude': -87.68405,
  'latitude': 41.897385,
  'restaurant_1': {'business_name': 'Janellie’s Kitchen',
   'categories_titles': ['Puerto Rican'],
   'rating': 5.0,
   'latitude': 41.90281613454157,
   'longitude': -87.69265859457461,
   'display_address': '2617 W Division St Chicago, IL 60622',
   'url': 'https://www.yelp.com/biz/janellie-s-kitchen-chicago-2?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q'},
  'restaurant_2': {'business_name': 'Healthy Substance Cantina',
   'categories_titles': ['Vegan'],
   'rating': 5.0,
   'latitude': 41.908631,
   'longitude': -87.67

### Mapping Properties and Restaurants Together

In [45]:
import folium

def create_property_restaurant_map(properties):
    # Attribution for custom tileset
    attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    
    # Initialize the map centered around Chicago
    property_map = folium.Map(location=[41.881832, -87.623177], zoom_start=11, tiles='OpenStreetMap', attr=attr)

    # Loop through each location in the list
    for prop in properties:
        # Extract property details
        latitude = prop['latitude']
        longitude = prop['longitude']
        detail_url = prop['detailUrl']
        address = prop['address']

        # Add a marker to the map for the property
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(f'<a href="{detail_url}" target="_blank">{address}</a>', max_width=250),
            tooltip="Click for property details",
            icon=folium.Icon(icon='home', color="blue")
        ).add_to(property_map)

        # Loop through the associated restaurants
        for key in ['restaurant_1', 'restaurant_2']:
            if key in prop:
                restaurant = prop[key]
                # Extract relevant restaurant data
                name = restaurant['business_name']
                categories = ', '.join(restaurant['categories_titles'])
                rating = restaurant['rating']
                res_latitude = restaurant['latitude']
                res_longitude = restaurant['longitude']
                res_address = restaurant['display_address']
                url = restaurant['url']

                # Add a marker to the map for the restaurant
                folium.Marker(
                    location=[res_latitude, res_longitude],
                    popup=folium.Popup(f'<a href="{url}" target="_blank">{name}</a><br/>{categories}<br/>Rating: {rating}', max_width=250),
                    tooltip=f"Click for details on {name}",
                    icon=folium.Icon(icon='cutlery', color="red")
                ).add_to(property_map)

    # Return the map
    return property_map

In [46]:
# Create the map
props_rests_map = create_property_restaurant_map(top_properties)

display(props_rests_map)
# Display or save the map
# my_map.save('map.html') or display it directly in a Jupyter notebook using `my_map`

### Example - Property details endpoint

#### Description

In [47]:
top_properties[1]['detailUrl']

'https://www.zillow.com/apartments/chicago-il/nema-chicago/9RwqTm/'

In [34]:
# Endpoint
url_details = "https://zillow-com1.p.rapidapi.com/property"

# URL example
property_url = top_properties[1]['detailUrl']

querystring = {"property_url":property_url}
response = requests.get(url_details, headers = headers, params = querystring)

In [35]:
response.json()

{'listingProvider': {'enhancedVideoURL': None,
  'showNoContactInfoMessage': False,
  'postingGroupName': None,
  'isZRMSourceText': None,
  'showLogos': None,
  'logos': {},
  'sourceText': 'Source: Zillow Rentals',
  'title': 'Listing Provided by',
  'disclaimerText': None,
  'postingWebsiteURL': None,
  'agentLicenseNumber': None,
  'postingWebsiteLinkText': 'See listing website',
  'enhancedDescriptionText': None,
  'agentName': None},
 'zpid': 158698100,
 'buildingPermits': None,
 'propertyTaxRate': 1.89,
 'contact_recipients': {},
 'zipcode': '60624',
 'openHouseSchedule': {},
 'longitude': -87.7079,
 'zestimateLowPercent': '11',
 'address': {'community': None,
  'city': 'Chicago',
  'state': 'IL',
  'neighborhood': None,
  'subdivision': None,
  'streetAddress': '3240 W Fulton Blvd',
  'zipcode': '60624'},
 'cityId': 17426,
 'timeOnZillow': '1 hour',
 'url': '/homedetails/3240-W-Fulton-Blvd-Chicago-IL-60624/158698100_zpid/',
 'zestimate': 300400,
 'imgSrc': 'https://photos.zillo

In [36]:
response = response.json()
response['description']

"Hello!\n\nThis rehabbed house has central heat & air, exposed bricks, dark hardwood floors, new stainless steel kitchen appliances, in-unit washer and dryer, and plenty of storage space in the basement. It also has a firepit in the backyard for bonfires. \n\nAbout the neighborhood:\nThis unfurnished house is in the heart of East Garfield Park, less than a five-minute walk from the Kedzie Green line and Metra stations, as well as the nearest 52 Kedzie, 82 Homan and 94 California bus stops. There is plenty of space downstairs for your bikes, so you can get around very easily from here without a car. If you do have a car though, you can have a spot out back for $25/month each, or use one of the parking spots on the street out front. \n\nThis house is within a few blocks of cool neighborhood spots like Passion House Coffee, Breakthrough Community Center, Inspiration Kitchens, and the Garfield Park Conservatory, and just a short bike ride away from everything in Humboldt Park, Ukranian Vil

In [48]:
def fetch_descriptions(properties):
    """
    Fetches the descriptions of each property and appends them to the list.

    Parameters:
    - properties: A list of dictionaries representing properties.

    Returns:
    - A list of dictionaries(the same format as input).
    """
    url_details = "https://zillow-com1.p.rapidapi.com/property"

    for prop in properties:
        # Fetch detailed information using detailUrl
        querystring = {"property_url": prop['detailUrl']}
        response = requests.get(url_details, headers=headers, params=querystring)

        # Convert to JSON and fetch description
        detail_data = response.json()
        prop['description'] = detail_data.get('description', 'No description available')

    return properties

In [38]:
top_properties = fetch_descriptions(top_properties)
top_properties

[{'propertyType': None,
  'address': '2611 N Hermitage Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/norweta/BXQZ9R/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/65ca2c82486f45afdc0e1f156f6b514f-p_e.jpg',
  'description': 'No description available'},
 {'propertyType': 'SINGLE_FAMILY',
  'address': '3240 W Fulton Blvd, Chicago, IL 60624',
  'price': 2800,
  'bedrooms': 5,
  'bathrooms': 2,
  'detailUrl': 'https://www.zillow.com/homedetails/3240-W-Fulton-Blvd-Chicago-IL-60624/158698100_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/402d1685a0321ebbd7c3b2ceaecb3822-p_e.jpg',
  'description': "Hello!\n\nThis rehabbed house has central heat & air, exposed bricks, dark hardwood floors, new stainless steel kitchen appliances, in-unit washer and dryer, and plenty of storage space in the basement. It also has a firepit in the backyard for bonfires. \n\nAbout the neighborhood:\nThis unfurnis

#### resoFacts

In [49]:
def fetch_resoFacts(properties, headers, keys_to_fetch):
    """
    Fetches specific information from the resoFacts dictionary of each property and appends it in a nested format.

    Parameters:
    - properties: A list of dictionaries representing properties.
    - headers: Headers for the API request.
    - keys_to_fetch: A list of keys to fetch from the resoFacts dictionary.

    Returns:
    - A list of dictionaries with the specified information appended in a nested format.
    """
    url_details = "https://zillow-com1.p.rapidapi.com/property"

    for prop in properties:
        # Fetch detailed information using detailUrl
        querystring = {"property_url": prop['detailUrl']}
        response = requests.get(url_details, headers=headers, params=querystring)
        detail_data = response.json()

        # Fetch information from the resoFacts dictionary
        resoFacts = detail_data.get('resoFacts', {})

        # Create a nested dictionary to store resoFacts information
        prop['resoFacts'] = {}

        for key in keys_to_fetch:
            prop['resoFacts'][key] = resoFacts.get(key, 'N/A')

    return properties

In [50]:
# key to fetch
keys_to_fetch_resoFacts = [
    'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
    'laundryFeatures', 'associationFee',
    'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
]

top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)
top_properties

#### Schools

In [51]:
def fetch_schools(properties, headers):
    """
    Fetches information about nearby schools and appends it in a nested format.

    Parameters:
    - properties: A list of dictionaries representing properties.
    - headers: Headers for the API request.

    Returns:
    - A list of dictionaries with school information appended in a nested format.
    """
    url_details = "https://zillow-com1.p.rapidapi.com/property"

    for prop in properties:
        # Fetch detailed information using detailUrl
        querystring = {"property_url": prop['detailUrl']}
        response = requests.get(url_details, headers=headers, params=querystring)
        detail_data = response.json()

        # Fetch schools information
        schools = detail_data.get('schools', [])

        # Create a nested dictionary to store school information
        prop['schools'] = schools

    return properties

In [42]:
top_properties = fetch_schools(top_properties, headers)
top_properties

[{'propertyType': None,
  'address': '2611 N Hermitage Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/norweta/BXQZ9R/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/65ca2c82486f45afdc0e1f156f6b514f-p_e.jpg',
  'description': 'No description available',
  'resoFacts': {'hasGarage': 'N/A',
   'hasPetsAllowed': 'N/A',
   'heating': 'N/A',
   'cooling': 'N/A',
   'flooring': 'N/A',
   'appliances': 'N/A',
   'laundryFeatures': 'N/A',
   'associationFee': 'N/A',
   'livingArea': 'N/A',
   'taxAnnualAmount': 'N/A',
   'parkingFeatures': 'N/A',
   'stories': 'N/A'},
  'schools': []},
 {'propertyType': 'SINGLE_FAMILY',
  'address': '3240 W Fulton Blvd, Chicago, IL 60624',
  'price': 2800,
  'bedrooms': 5,
  'bathrooms': 2,
  'detailUrl': 'https://www.zillow.com/homedetails/3240-W-Fulton-Blvd-Chicago-IL-60624/158698100_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/402d1685a0321ebbd7c3b2ceaecb

### For main use case

In [52]:
import json

In [53]:
user_query = "I am looking for a two-bedroom apartment for less than $2000 per month in Hyde Park. Are there good italian restaurants in Hyde Park?"
prompt = generate_prompt_apifilter(instruction, user_query)
print(get_chat_response(chat, prompt))

{
"location":"chicago, il",
"bedsMin":"2",
"bedsMax":"2", 
"status_type":"ForRent",
"rentMaxPrice":"2000",
"keywords":"Hyde Park, italian",
"sort":"Newest"
}


In [54]:
# get an api filter through Gemini
querystring = get_chat_response(chat, prompt)

# from str to dict
querystring = json.loads(querystring)

# get listings from zillow api
response = requests.get(url, headers = headers, params = querystring)

In [55]:
# Define fields to extract
fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]

# Get top 3 properties with selected fields
top_properties = extract_properties(response, 3, fields)
top_properties

[{'propertyType': None,
  'address': '1501 N California Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/b/1501-n.-california-ave.-chicago-il-CJcL/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/f08d86d33bb00e30755fba122ce1f4f2-p_e.jpg',
  'longitude': -87.69659,
  'latitude': 41.908566},
 {'propertyType': None,
  'address': '2701 S Indiana Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/eden-commons/5XjNY3/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/c533fb4ba892997a014b728e64db7108-p_e.jpg',
  'longitude': -87.62091,
  'latitude': 41.844006},
 {'propertyType': 'APARTMENT',
  'address': '938 W 34th St, Chicago, IL 60608',
  'price': 1300,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/938-W-34th-St-Chicago-IL-60608/2103556011_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/39

In [56]:
# get description
top_properties = fetch_descriptions(top_properties)

# get detail info from resoFacts
keys_to_fetch_resoFacts = [
    'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
    'laundryFeatures', 'associationFee',
    'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
]
top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)
# get school info
top_properties = fetch_schools(top_properties, headers)
top_properties

[{'propertyType': None,
  'address': '1501 N California Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/b/1501-n.-california-ave.-chicago-il-CJcL/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/f08d86d33bb00e30755fba122ce1f4f2-p_e.jpg',
  'longitude': -87.69659,
  'latitude': 41.908566,
  'description': 'No description available',
  'resoFacts': {'hasGarage': 'N/A',
   'hasPetsAllowed': 'N/A',
   'heating': 'N/A',
   'cooling': 'N/A',
   'flooring': 'N/A',
   'appliances': 'N/A',
   'laundryFeatures': 'N/A',
   'associationFee': 'N/A',
   'livingArea': 'N/A',
   'taxAnnualAmount': 'N/A',
   'parkingFeatures': 'N/A',
   'stories': 'N/A'},
  'schools': []},
 {'propertyType': None,
  'address': '2701 S Indiana Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/apartments/chicago-il/eden-commons/5XjNY3/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/c53

In [57]:
# Call Yelp API using top properties
yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, chat)

fields_to_extract = ['business_name', 'categories_titles', 'rating', 'latitude', 'longitude', 'display_address', 'url']

top_restaurants = extract_business_info(yelp_results, 2, fields_to_extract)

# Add restaurant info to property info
top_properties = merge_property_and_restaurant_info(top_properties, top_restaurants)
top_properties

Category determined: italian
{'location': '41.908566, -87.69659', 'term': 'restaurants', 'radius': 2000, 'categories': 'italian', 'sort_by': 'rating', 'limit': 2}
{'location': '41.844006, -87.62091', 'term': 'restaurants', 'radius': 2000, 'categories': 'italian', 'sort_by': 'rating', 'limit': 2}
{'location': '41.833042, -87.65012', 'term': 'restaurants', 'radius': 2000, 'categories': 'italian', 'sort_by': 'rating', 'limit': 2}


[{'propertyType': None,
  'address': '1501 N California Ave, Chicago, IL',
  'price': None,
  'bedrooms': None,
  'bathrooms': None,
  'detailUrl': 'https://www.zillow.com/b/1501-n.-california-ave.-chicago-il-CJcL/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/f08d86d33bb00e30755fba122ce1f4f2-p_e.jpg',
  'longitude': -87.69659,
  'latitude': 41.908566,
  'description': 'No description available',
  'resoFacts': {'hasGarage': 'N/A',
   'hasPetsAllowed': 'N/A',
   'heating': 'N/A',
   'cooling': 'N/A',
   'flooring': 'N/A',
   'appliances': 'N/A',
   'laundryFeatures': 'N/A',
   'associationFee': 'N/A',
   'livingArea': 'N/A',
   'taxAnnualAmount': 'N/A',
   'parkingFeatures': 'N/A',
   'stories': 'N/A'},
  'schools': [],
  'restaurant_1': {'business_name': 'Antico',
   'categories_titles': ['Italian'],
   'rating': 4.4,
   'latitude': 41.91746,
   'longitude': -87.68281,
   'display_address': '1946 N Leavitt St Chicago, IL 60647',
   'url': 'https://www.yelp.com/biz/antico-chicago?a

In [58]:
# Create property map
property_map = create_property_map(top_properties)

property_map

In [59]:
# Create property restaurant map
property_restaurant_map = create_property_restaurant_map(top_properties)

property_restaurant_map

## 4. Generate advice with the property listings

In [60]:
def generate_prompt_property(instruction, user_query, property_info, fields):
    updated_instruction = instruction.replace("{USER_QUERY}", user_query)
    updated_instruction = updated_instruction.replace("{PROPERTY_INFO}", property_info)
    updated_instruction = updated_instruction.replace("{KEY_FIELDS}", fields)
    return updated_instruction

In [61]:
instruction = """
### Instructions ###
I want you to act as a real estate advisor. Please recommend properties to the customer. 
'User’s query' represents the questions you are asked by the customer. 
'Real estate properties to present to the user' are the properties you will introduce to the customer. 

Note1: Include all information from {KEY_FIELDS}, including links. 
Note2: The "Resofacts" and "Schools" fields are in JSON format. You do not need to include all the information from "Resofacts" and "Schools" in your initial response. 
However, if the user asks specific questions about "Resofacts" and "Schools", include relevant information in your answer.

### User’s query ###
{USER_QUERY}

### Real estate properties to present to the user ###
{PROPERTY_INFO}

### Output ###
"""

In [62]:
def format_properties(properties, fields):
    formatted_properties = []
    for prop in properties:
        # Format and concatenate each field of each property
        formatted_prop = ', '.join(f"{field.capitalize().replace('_', ' ')}: {prop.get(field, 'N/A')}" for field in fields)
        formatted_properties.append(formatted_prop)
    return '\n\n'.join(formatted_properties)

In [63]:
# Change format
fields_desc = fields + ['description', 'resoFacts', 'schools', 'restaurant_1', 'restaurant_2']
property_info = format_properties(top_properties, fields_desc)
print(property_info)

Propertytype: None, Address: 1501 N California Ave, Chicago, IL, Price: None, Bedrooms: None, Bathrooms: None, Detailurl: https://www.zillow.com/b/1501-n.-california-ave.-chicago-il-CJcL/, Imgsrc: https://photos.zillowstatic.com/fp/f08d86d33bb00e30755fba122ce1f4f2-p_e.jpg, Longitude: -87.69659, Latitude: 41.908566, Description: No description available, Resofacts: {'hasGarage': 'N/A', 'hasPetsAllowed': 'N/A', 'heating': 'N/A', 'cooling': 'N/A', 'flooring': 'N/A', 'appliances': 'N/A', 'laundryFeatures': 'N/A', 'associationFee': 'N/A', 'livingArea': 'N/A', 'taxAnnualAmount': 'N/A', 'parkingFeatures': 'N/A', 'stories': 'N/A'}, Schools: [], Restaurant 1: {'business_name': 'Antico', 'categories_titles': ['Italian'], 'rating': 4.4, 'latitude': 41.91746, 'longitude': -87.68281, 'display_address': '1946 N Leavitt St Chicago, IL 60647', 'url': 'https://www.yelp.com/biz/antico-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgX

In [64]:
# generate prompt
prompt = generate_prompt_property(instruction, user_query, property_info, str(fields))

In [65]:
print(prompt)


### Instructions ###
I want you to act as a real estate advisor. Please recommend properties to the customer. 
'User’s query' represents the questions you are asked by the customer. 
'Real estate properties to present to the user' are the properties you will introduce to the customer. 

Note1: Include all information from ['propertyType', 'address', 'price', 'bedrooms', 'bathrooms', 'detailUrl', 'imgSrc', 'longitude', 'latitude'], including links. 
Note2: The "Resofacts" and "Schools" fields are in JSON format. You do not need to include all the information from "Resofacts" and "Schools" in your initial response. 
However, if the user asks specific questions about "Resofacts" and "Schools", include relevant information in your answer.

### User’s query ###
I am looking for a two-bedroom apartment for less than $2000 per month in Hyde Park. Are there good italian restaurants in Hyde Park?

### Real estate properties to present to the user ###
Propertytype: None, Address: 1501 N Califor

In [66]:
# get final response from Gemini
print(get_chat_response(chat, prompt))
display(property_restaurant_map)

## Hyde Park Apartment Recommendations

Hi there! Based on your request for a two-bedroom apartment in Hyde Park for under $2000 a month, I've found several options that might interest you:

**Option 1:**

- Address: **1501 N California Ave, Chicago, IL**
- Price: **Data unavailable**
- Bedrooms: **Data unavailable**
- Bathrooms: **Data unavailable**
- Link: https://www.zillow.com/b/1501-n.-california-ave.-chicago-il-CJcL/
- Image: https://photos.zillowstatic.com/fp/f08d86d33bb00e30755fba122ce1f4f2-p_e.jpg
- Description: No description available
- Resofacts: Unavailable
- Schools: Unavailable
- Restaurants nearby:
    - Antico (Italian, 4.4 stars): https://www.yelp.com/biz/antico-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GgXKz4qBvfURJxuhy6g81Q
    - Segnatore (Italian, 4.4 stars): https://www.yelp.com/biz/segnatore-chicago?adjust_creative=GgXKz4qBvfURJxuhy6g81Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business

In [73]:
prompt_2 = "I want more details about the third property"
print(get_chat_response(chat, prompt_2))

## 3 Bedroom Detached House, [Address]

**Key features:**

* 3 bedrooms, 2 bathrooms
* Large living room with fireplace
* Modern kitchen with granite countertops and stainless steel appliances
* Spacious backyard with deck
* 2-car garage
* Located in a quiet neighborhood close to schools and parks

**Additional details:**

* The house was built in [Year] and has been well-maintained.
* It has central air conditioning and heating.
* The master bedroom has a walk-in closet and a private bathroom.
* The backyard is fenced in and has a patio.
* The house is located on a cul-de-sac.

**Price:**

* The asking price is [Price].

**Photos:**

* You can view photos of the property here: [Link to photos]

**Availability:**

* The property is available for immediate occupancy.

**Contact:**

* Please contact [Name] at [Phone number] or [Email address] to schedule a showing.

**Additional information:**

* The property is located in a great school district.
* The neighborhood is very safe and fami

In [74]:
prompt_3 = "Is this property close to public transportation facilities?"
print(get_chat_response(chat, prompt_3))

I apologize, but I don't have access to real-time information like public transportation schedules or routes. However, I can help you find this information online. Here are some resources:

* **[City/Town Name] Public Transportation Website:** https://[city/town name] .gov/public-transportation/
* **Google Maps:** https://www.google.com/maps/
* **Moovit:** https://moovitapp.com/

To find out if the property is close to public transportation facilities, you can use these resources to search for public transportation stops and routes near the property address. You can also use the "Directions" feature on Google Maps to see how long it would take to get to the nearest bus stop, train station, or subway station by public transportation.

I hope this information is helpful!


## 5. Integrating all functions

In [2]:
## TBD